In [3]:
%%sh
pip install -qU httpx[http2] yfinance openai

In [2]:
import httpx
import json
import pprint
import yfinance
from openai import OpenAI
from api_key import api_key

endpoint="https://models.arcee.ai/v1"

client = OpenAI(
    base_url=endpoint,
    api_key=api_key,
    http_client=httpx.Client(http2=True)
)

In [3]:
def get_stock_price(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    price = stock.history(period="1d")["Close"].values[0]
    return f"The last closing price of {company_name} ({stock_symbol}) was ${price:.2f}."


def get_ceo_name(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    info = stock.info
    ceo = info['companyOfficers'][0]['name']
    return f"The CEO of {company_name} is {ceo}. The full job title is {info['companyOfficers'][0]['title']}."


def get_company_summary(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    summary = stock.info['longBusinessSummary']
    return summary

In [ ]:
get_stock_price("Google", "GOOGL")

In [ ]:
get_ceo_name("Ford", "F")

In [ ]:
get_company_summary("Verizon", "VZ")

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Use this function to get the last price of a stock",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "A company name (e.g., Mc Donalds)",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "A company stock ticker (e.g., MCD)",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_ceo_name",
            "description": "Use this function to get the name of a company's CEO",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "A company name (e.g., Mc Donalds)",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "A company stock ticker (e.g., MCD)",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_company_summary",
            "description": "Use this function to describe a company's activities, products, services, and customers",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "A company name (e.g., Mc Donalds)",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "A company stock ticker (e.g., MCD)",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    }
]

In [8]:
def call_tools(user_prompt, max_tokens=128):
    response = client.chat.completions.create(
        model="caller",
        messages=[

            {
                "role": "user",
                "content": user_prompt
            }
        ],
        tools=tools,
        tool_choice="auto",
        max_tokens=max_tokens,
    )
    
    tool_calls = response.choices[0].message.tool_calls

    # Check if there are any tool calls
    if tool_calls:
        # Extract the first tool call (assuming there's at least one)
        first_tool_call = tool_calls[0]
        
        # Extract function name and arguments
        function_name = first_tool_call.function.name
        arguments_json = first_tool_call.function.arguments
        arguments_dict = json.loads(arguments_json)
        
        ## Assuming the function is in the current namespace or imported
        if function_name in globals():
            # Get the function object based on its name
            function_to_call = globals()[function_name]
            
            print(f"Calling {function_name} with arguments: {arguments_dict}")
            
            # Call the function with unpacked keyword arguments
            result = function_to_call(**arguments_dict)
            return result
        else:
            print(f"Function {function_name} not found in the global namespace.")
            return None
    else:
        # No tool call: print the generated response
        print("No tool call")
        return None


In [17]:
#user_prompt = "Who's the CEO of General Motors?"
#user_prompt = "What's the last price of Mc Donalds?"
user_prompt = "What's the main business of 3M?"
#user_prompt = "Where are the main offices of 3M?"
#user_prompt = "What's the population of the capital of New Zealand?"
#user_prompt = "Is Lamb Weston the largest producer of french fries?"

In [ ]:
response = call_tools(user_prompt)
pprint.pprint(response)

In [21]:
def call_tools_and_invoke_model(user_prompt, max_tokens=512):
    tool_result = call_tools(user_prompt)
    
    response = client.chat.completions.create(
        model="virtuoso-small",
        messages=[
            {
                "role": "system",
                "content": "You are a knowledgeable financial assistant giving structured and detailed answers. Use a business-oriented tone."
            },
            {
                "role": "user",
                "content": f"""Answer the following question: {user_prompt} using the tool result: {tool_result}.
                If the tool result is empty or not useful, say it is not useful and answer the question without using the tool result.
                If the tool result is useful, you can augment it with your own knowledge as long as it's not contradictory.
                """
            }
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content


In [ ]:
response = call_tools_and_invoke_model(user_prompt)
pprint.pprint(response)
